In [1]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 6.5 MB/s eta 0:00:00


In [2]:
pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.1 MB/s eta 0:00:00


In [3]:
pip install optuna-integration

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 3.0 MB/s eta 0:00:00


In [4]:
import numpy as np
import optuna
from optuna.integration import CatBoostPruningCallback

from catboost import CatBoostRegressor
import catboost as cb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.preprocessing import PolynomialFeatures

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
test=pd.read_csv('/content/drive/MyDrive/kag fellow/test_final.csv',index_col=0)
train=pd.read_csv('/content/drive/MyDrive/kag fellow/train_imputed_rf.csv',index_col=0)

In [7]:
X=train.drop(['price','id'],axis=1)

y=train['price']

In [8]:
# Initialize PolynomialFeatures
poly = PolynomialFeatures(degree=2, include_bias=False)
columns=['power', 'engine_size', 'engine_cylinders','milage','model_year']

In [9]:
# Fit and transform the data
poly_features = poly.fit_transform(X[columns])
poly_feature_names = poly.get_feature_names_out(X[columns].columns)
poly_df = pd.DataFrame(poly_features, columns=poly_feature_names)

In [10]:
# Drop the original columns that were transformed
data_dropped = X.drop(columns=columns)

# Combine the transformed columns with the rest of the dataset
final_df = pd.concat([data_dropped, poly_df], axis=1)

In [11]:
X=final_df.drop(['mil_sq'],axis=1)

In [12]:
X['price']=y.values

In [13]:
train=X
TARGET='price'
FEATURES=X.drop('price',axis=1).columns


In [15]:
N_split = 2
kf = KFold(n_splits=N_split)

import optuna

def objective(trial):
    params = {
                #'iterations' : 10000, replaced by early stopping
                'eval_metric': 'RMSE',
                'loss_function':'RMSE',
                'use_best_model': True,
                'random_seed' : 1,
                'learning_rate' :trial.suggest_float('learning_rate', 0.01, 0.15,log=True),
                "depth": trial.suggest_int("depth", 1, 8),
                'l2_leaf_reg' :trial.suggest_loguniform('l2_leaf_reg', 1e-8, 20),
                'random_strength' : trial.suggest_loguniform('random_strength', 1, 50),
                'grow_policy':trial.suggest_categorical ('grow_policy', ['Lossguide','SymmetricTree']),
                'max_bin': trial.suggest_int("max_bin", 20, 500),
                'min_data_in_leaf':trial.suggest_int('min_data_in_leaf', 1, 100),
                "boosting_type": trial.suggest_categorical("boosting_type", [ "Plain"]),
                "bootstrap_type": trial.suggest_categorical(
                "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"])
            }

    if params['grow_policy'] == 'Lossguide':
        params['max_leaves']:trial.suggest_int('max_leaves', 1, 100)
    if params["bootstrap_type"] == "Bayesian":
        params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif params["bootstrap_type"] == "Bernoulli":
        params["subsample"] = trial.suggest_float("subsample", 0.1, 1)


    score_list = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(train, train[TARGET])):
        X_tr = train.loc[train_idx][FEATURES]
        X_va = train.iloc[val_idx][FEATURES]

        # Preprocess the data
        X_tr_f = X_tr # Use the correct preprocessing if required (normalization...)
        y_tr = train[TARGET].loc[train_idx].values

        X_va_f = X_va # Use the correct preprocessing if required (normalization...)
        y_va = train[TARGET].loc[val_idx].values

        # Train the model
        model = CatBoostRegressor(**params)
        pruning_callback = CatBoostPruningCallback(trial, "RMSE")
        model.fit(
                X_tr_f,
                y_tr,
                eval_set =[( X_va_f,y_va)],
                verbose =0,
                early_stopping_rounds = 100,
                callbacks=[pruning_callback]
                )
         # evoke pruning manually.
        pruning_callback.check_pruned()

        # Predictions
        y_va_pred = model.predict(X_va_f)
        score = mean_squared_error(y_va, y_va_pred,squared = False)
        score_list.append(score)

    return sum(score_list) / len(score_list)
if __name__ == "__main__":
    study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), direction="minimize"
    )
    study.optimize(objective, n_trials=300,timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2024-06-26 22:53:33,774] A new study created in memory with name: no-name-98849042-3b3a-4d0f-95d7-aeace5ffcaf9
[I 2024-06-26 22:53:47,173] Trial 0 finished with value: 68369.54052002827 and parameters: {'learning_rate': 0.0934636134591269, 'depth': 5, 'l2_leaf_reg': 11.03927488036102, 'random_strength': 8.298808630912072, 'grow_policy': 'Lossguide', 'max_bin': 357, 'min_data_in_leaf': 4, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS'}. Best is trial 0 with value: 68369.54052002827.
[I 2024-06-26 22:53:54,877] Trial 1 finished with value: 68659.69329431241 and parameters: {'learning_rate': 0.1435136266323533, 'depth': 3, 'l2_leaf_reg': 5.2393293545466525e-06, 'random_strength': 1.9705619527474019, 'grow_policy': 'Lossguide', 'max_bin': 242, 'min_data_in_leaf': 32, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 5.624524854772448}. Best is trial 0 with value: 68369.54052002827.
[I 2024-06-26 22:54:13,837] Trial 2 finished with value: 68599.47232481383

Number of finished trials: 253
Best trial:
  Value: 68208.84489966124
  Params: 
    learning_rate: 0.14442874023057728
    depth: 8
    l2_leaf_reg: 19.65271385483249
    random_strength: 12.701816999382666
    grow_policy: SymmetricTree
    max_bin: 187
    min_data_in_leaf: 6
    boosting_type: Plain
    bootstrap_type: MVS
